In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

# multiple contexts 허용
conf = SparkConf()
conf.set("spark.driver.allowMultipleContexts", "true")

sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write").config(conf=conf).getOrCreate()

#sc = SparkContext.getOrCreate(SparkConf())
#spark = SparkSession.builder.master('local').appName("morphy_test111_dgdfgdf").config(conf=conf).getOrCreate()
#sqlContext = SQLContext(sparkContext=sc.sparkContext, sparkSession=sc)

In [3]:
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StringType,IntegerType
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *


In [4]:
#data load
song_df = sparkSession.read.csv('hdfs://192.168.0.17:9000/user/hadoop/data.csv', header = False)

#header 설정
song_df = song_df.selectExpr("_c0 as year", "_c1 as month", "_c2 as rank", "_c3 as title", 
                   "_c4 as artist", "_c5 as album", "_c6 as genre", "_c8 as likes", "_c9 as keywords" )

song_df.show(10,400)

+----+-----+----+---------+------+----------------+-----+------+-----------------------------------------+
|year|month|rank|    title|artist|           album|genre| likes|                                 keywords|
+----+-----+----+---------+------+----------------+-----+------+-----------------------------------------+
|2011|    1|   1|     좋은 날|   아이유|            Real|   댄스|134776|      ['말', 'beautiful', '위', '하늘', '고개']|
|2011|    1|   2| Oh Yeah |GD&TOP|          GD&TOP| 랩/힙합| 29231|           ['oh', 'yeah', '밤', '오늘', '별']|
|2011|    1|   3|High High|GD&TOP|          GD&TOP| 랩/힙합| 28998|        ['ha', 'high', 'fly', '반항', '이유']|
|2011|    1|   4|     니까짓게|   씨스타|            니까짓게|   댄스| 11969|['baby', '사랑', 'sorry', 'Sistar', 'Baby']|
|2011|    1|   5|     샤이보이|   시크릿|            샤이보이|   댄스| 16615|       ['Shy', 'Boy', 'Time', '밥', 'Say']|
|2011|    1|   6| 사랑이 고프다 |   박효신|   Gift (Part.2)|  발라드| 26964|               ['밖', '혼자', '날', '품', '안']|
|2011|    1|   7|    나쁜 남자|   JOO|   

In [5]:
def select_keyword(keywords,i):
    keywords = keywords.replace("[","").replace("]","").replace("'","").replace(" ","").split(",")
    if len(keywords)-1 < i:
        return ""
    else:
        return keywords[i]

In [6]:
#장르별 키워드 개수

schema = StructType([
    StructField("genre",StringType(),True),
    StructField("keyword",StringType(),True),
    StructField("title",StringType(),True)
    
])

key_genre_df= spark.createDataFrame(sc.emptyRDD(), schema)

# 전체 키워드 뽑기
for i in range(5):
    tmp_rdd = song_df.rdd.map( lambda x : Row( title = x["title"], genre = x["genre"].replace(" ","").split(",")[0] 
                                              , keyword = select_keyword(x["keywords"],i) ))
    tmp_df = spark.createDataFrame(tmp_rdd)
    key_genre_df = key_genre_df.union(tmp_df)

#중복 제거
key_genre_df = key_genre_df.dropDuplicates(['title', 'keyword'])
key_genre_df.show()

#장르 별 keyword 개수
genre_count = key_genre_df.groupBy(key_genre_df["genre"], 
                                   key_genre_df["keyword"]).count()

#빈 문자열 제거, 내림차순 정렬
genre_count = genre_count.filter("keyword != ''").sort('count', ascending=False)

genre_count.show(100)


+--------+--------+-----------------+
|   genre| keyword|            title|
+--------+--------+-----------------+
|    랩/힙합|       앞|         3호선 매봉역 |
|    록/메탈|  Bounce|           Bounce|
|      댄스|       때|          Breathe|
|     발라드|      조금|     Brothers＆Me |
|    랩/힙합|     BYE|           Bye U |
|R&B/Soul|      마음|  Bye bye my blue|
|      댄스|      마디|         Catch Me|
|     발라드|   count|     Count On Me |
|      댄스|       위|        Delicious|
|      댄스|       속|          Fiction|
|      댄스|       맘|        Flashback|
|      댄스|      GG|           GG Be |
|R&B/Soul|      그때|            Ghost|
|    랩/힙합|       몸|Girls Girls Girls|
|      댄스|      사랑|       Good Love |
|    랩/힙합|hesitate|    Hello Bitches|
|     발라드|      하루|          Holiday|
|      댄스|    look|How You Like That|
|R&B/Soul|      머리|          JUST U |
|     발라드|      세월|    Kiss The Rain|
+--------+--------+-----------------+
only showing top 20 rows

+--------+-------+-----+
|   genre|keyword|count|
+--------+--

In [7]:
#시대별 키워드 개수(2010년대 초(11~13) , 중(14~16) , 후반(17~19)) 

schema = StructType([
    StructField("age",StringType(),True),
    StructField("keyword",StringType(),True),
    StructField("title",StringType(),True)
])

key_age_df= spark.createDataFrame(sc.emptyRDD(), schema)

#3년 단위로 묶기
def set_age(x):
    x = int(x)
    if 2011 <= x <= 2013:
        return "2011-2013"
    elif 2014<= x <= 2016:
        return "2014-2016"
    elif 2017 <= x <= 2019:
        return "2017-2019"
    else:
        return ""

# 전체 키워드 뽑기
for i in range(5):
    tmp_rdd = song_df.rdd.map( lambda x : Row( title = x["title"], age = set_age(x["year"]) , keyword = select_keyword(x["keywords"],i) ))
    tmp_df = spark.createDataFrame(tmp_rdd)
    key_age_df = key_age_df.union(tmp_df)

key_age_df.show()

#시대별 keyword 개수
age_count = key_age_df.groupBy(key_age_df["age"], key_age_df["keyword"]).count()

#빈 문자열 제거, 내림차순 정렬
age_count = age_count.filter("keyword != ''").filter("age != ''").sort('count', ascending=False)

age_count.show(100)


+---------+-------+----------------+
|      age|keyword|           title|
+---------+-------+----------------+
|2011-2013|      말|            좋은 날|
|2011-2013|     oh|        Oh Yeah |
|2011-2013|     ha|       High High|
|2011-2013|   baby|            니까짓게|
|2011-2013|    Shy|            샤이보이|
|2011-2013|      밖|        사랑이 고프다 |
|2011-2013|     남자|           나쁜 남자|
|2011-2013|       |            뻑이가요|
|2011-2013|  Knock|           왜 이러니|
|2011-2013|     Ah|          yayaya|
|2011-2013|     하루|        니가 따끔거려서|
|2011-2013|     사랑|          가지 말아요|
|2011-2013|      꿈|              왜 |
|2011-2013|   오랫동안|            똑같다면|
|2011-2013|      별|             별처럼|
|2011-2013|       |          이게 아닌데|
|2011-2013|     사이|처음 사랑하는 연인들을 위해 |
|2011-2013|    don|      Bad Temper|
|2011-2013| Jumpin|             점핑 |
|2011-2013|       |          집에 가지마|
+---------+-------+----------------+
only showing top 20 rows

+---------+-------+-----+
|      age|keyword|count|
+---------+-------+-----+
|2011-20

In [8]:
#연도별 키워드 개수 (2011~2020)


schema = StructType([
    StructField("keyword",StringType(),True),
    StructField("title",StringType(),True),
    StructField("year",StringType(),True)
    
])

key_year_df= spark.createDataFrame(sc.emptyRDD(), schema)

# 키워드 뽑기
for i in range(5):
    tmp_rdd = song_df.rdd.map( lambda x : Row( title = x["title"], year = x["year"] , keyword = select_keyword(x["keywords"],i) ))
    tmp_df = spark.createDataFrame(tmp_rdd)
    key_year_df = key_year_df.union(tmp_df)

key_year_df.show()
#연도별 keyword 개수
one_year_count = key_year_df.groupBy(key_year_df["year"], key_year_df["keyword"]).count()

#빈 문자열 제거, 내림차순 정렬
one_year_count = one_year_count.filter("keyword != ''").sort('count', ascending=False)

one_year_count.show(100)


+-------+----------------+----+
|keyword|           title|year|
+-------+----------------+----+
|      말|            좋은 날|2011|
|     oh|        Oh Yeah |2011|
|     ha|       High High|2011|
|   baby|            니까짓게|2011|
|    Shy|            샤이보이|2011|
|      밖|        사랑이 고프다 |2011|
|     남자|           나쁜 남자|2011|
|       |            뻑이가요|2011|
|  Knock|           왜 이러니|2011|
|     Ah|          yayaya|2011|
|     하루|        니가 따끔거려서|2011|
|     사랑|          가지 말아요|2011|
|      꿈|              왜 |2011|
|   오랫동안|            똑같다면|2011|
|      별|             별처럼|2011|
|       |          이게 아닌데|2011|
|     사이|처음 사랑하는 연인들을 위해 |2011|
|    don|      Bad Temper|2011|
| Jumpin|             점핑 |2011|
|       |          집에 가지마|2011|
+-------+----------------+----+
only showing top 20 rows

+----+-------+-----+
|year|keyword|count|
+----+-------+-----+
|2011|     사랑|  362|
|2012|     사랑|  300|
|2013|     사랑|  279|
|2018|      날|  267|
|2019|     사랑|  255|
|2014|     사랑|  238|
|2018|     사랑|  2

In [9]:
# 계절별 키워드

schema = StructType([
    StructField("keyword",StringType(),True),
    StructField("season",StringType(),True),
    StructField("title",StringType(),True)
])


key_season_df= spark.createDataFrame(sc.emptyRDD(), schema)

#계절별로 나누기
def set_season(x):
    x = int(x)
    if 3 <= x <= 5:
        return "spring"
    elif 6<= x <=8 :
        return "summer"
    elif 9 <= x <= 11:
        return "fall"
    else:
        return "winter"

# 전체 키워드 뽑기
for i in range(5):
    tmp_rdd = song_df.rdd.map( lambda x : Row( title = x["title"], season = set_season(x["month"]) , keyword = select_keyword(x["keywords"],i) ))
    tmp_df = spark.createDataFrame(tmp_rdd)
    key_season_df = key_season_df.union(tmp_df)

key_season_df.show()
#계절별 keyword 개수
season_count = key_season_df.groupBy(key_season_df["season"], key_season_df["keyword"]).count()

#빈 문자열 제거, 내림차순 정렬
season_count = season_count.filter("keyword != ''").sort('count', ascending=False)

season_count.show(100)




+-------+------+----------------+
|keyword|season|           title|
+-------+------+----------------+
|      말|winter|            좋은 날|
|     oh|winter|        Oh Yeah |
|     ha|winter|       High High|
|   baby|winter|            니까짓게|
|    Shy|winter|            샤이보이|
|      밖|winter|        사랑이 고프다 |
|     남자|winter|           나쁜 남자|
|       |winter|            뻑이가요|
|  Knock|winter|           왜 이러니|
|     Ah|winter|          yayaya|
|     하루|winter|        니가 따끔거려서|
|     사랑|winter|          가지 말아요|
|      꿈|winter|              왜 |
|   오랫동안|winter|            똑같다면|
|      별|winter|             별처럼|
|       |winter|          이게 아닌데|
|     사이|winter|처음 사랑하는 연인들을 위해 |
|    don|winter|      Bad Temper|
| Jumpin|winter|             점핑 |
|       |winter|          집에 가지마|
+-------+------+----------------+
only showing top 20 rows

+------+-------+-----+
|season|keyword|count|
+------+-------+-----+
|spring|     사랑|  667|
|winter|     사랑|  645|
|summer|     사랑|  559|
|  fall|     사랑|  54

In [9]:
import json

def rec_to_actions(df, INDEX, TYPE, cnt):
    idx = cnt
    print(idx)
    lst = []
    for record in df.to_dict(orient="records"):
        lst.append(('{ "index" : { "_index" : "%s", "_type" : "%s", "_id" : %d }}'% (INDEX, TYPE,idx)))
        lst.append(json.dumps(record, default=str, ensure_ascii=False))
        idx += 1
    return lst


    


In [11]:
# 장르별 키워드 json 파일에 저장

cnt = 0

result = rec_to_actions(genre_count.toPandas(),"keyword_by_genre", "k_b_g", cnt)
    
dest_file = 'elk_keyword/keyword_by_genre.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")
    
    
# 시대별 키워드 json 파일에 저장

cnt =0

result = rec_to_actions(age_count.toPandas(),"keyword_by_age", "k_b_a", cnt)

dest_file = 'elk_keyword/keyword_by_age.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")
    
# 연도별 키워드 json 파일에 저장

cnt =0

result = rec_to_actions(one_year_count.toPandas(),"keyword_by_year", "k_b_y", cnt)
    
dest_file = 'elk_keyword/keyword_by_year.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")

# 계절별 키워드 json 파일에 저장
result = rec_to_actions(season_count.toPandas(),"keyword_by_season", "k_b_s", cnt)
    
dest_file = 'elk_keyword/keyword_by_season.json'
output_file = open(dest_file, 'w', encoding='utf-8')
for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")


0
{'index': {'_id': 0, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사랑', 'count': 512, 'genre': '발라드'}
{'index': {'_id': 1, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '말', 'count': 322, 'genre': '발라드'}
{'index': {'_id': 2, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '날', 'count': 301, 'genre': '발라드'}
{'index': {'_id': 3, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사람', 'count': 151, 'genre': '발라드'}
{'index': {'_id': 4, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '눈물', 'count': 141, 'genre': '발라드'}
{'index': {'_id': 5, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '때', 'count': 131, 'genre': '발라드'}
{'index': {'_id': 6, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사랑', 'count': 128, 'genre': '댄스'}
{'index': {'_id': 7, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '맘', 'count': 122, 'genre': '발라드'}
{'index': {'_id': 8, '_type': 'k_b_g', '_index': 'keyword_b

{'index': {'_id': 334, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '옆', 'count': 9, 'genre': '발라드'}
{'index': {'_id': 335, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '흔적', 'count': 9, 'genre': '발라드'}
{'index': {'_id': 336, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '남', 'count': 9, 'genre': '댄스'}
{'index': {'_id': 337, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '빛', 'count': 9, 'genre': '발라드'}
{'index': {'_id': 338, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '입', 'count': 9, 'genre': '발라드'}
{'index': {'_id': 339, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'wan', 'count': 9, 'genre': 'R&B/Soul'}
{'index': {'_id': 340, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '기대', 'count': 9, 'genre': '발라드'}
{'index': {'_id': 341, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '어둠', 'count': 9, 'genre': '댄스'}
{'index': {'_id': 342, '_type': 'k_b_g', '_index': 'keyw

{'keyword': 'call', 'count': 5, 'genre': '랩/힙합'}
{'index': {'_id': 629, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'way', 'count': 5, 'genre': '랩/힙합'}
{'index': {'_id': 630, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'body', 'count': 5, 'genre': '댄스'}
{'index': {'_id': 631, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '거짓말', 'count': 5, 'genre': 'R&B/Soul'}
{'index': {'_id': 632, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사이', 'count': 5, 'genre': 'R&B/Soul'}
{'index': {'_id': 633, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'star', 'count': 5, 'genre': '랩/힙합'}
{'index': {'_id': 634, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '해', 'count': 5, 'genre': '발라드'}
{'index': {'_id': 635, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '참', 'count': 5, 'genre': '발라드'}
{'index': {'_id': 636, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '커피', 'count': 5, 'genre':

{'keyword': '입맞춤', 'count': 3, 'genre': '발라드'}
{'index': {'_id': 930, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '진실', 'count': 3, 'genre': '랩/힙합'}
{'index': {'_id': 931, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'matter', 'count': 3, 'genre': '댄스'}
{'index': {'_id': 932, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '인연', 'count': 3, 'genre': '록/메탈'}
{'index': {'_id': 933, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '한잔', 'count': 3, 'genre': '랩/힙합'}
{'index': {'_id': 934, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '안녕', 'count': 3, 'genre': '랩/힙합'}
{'index': {'_id': 935, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '상처', 'count': 3, 'genre': 'R&B/Soul'}
{'index': {'_id': 936, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '느낌', 'count': 3, 'genre': '록/메탈'}
{'index': {'_id': 937, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'light', 'count': 3, 'genre': 

{'index': {'_id': 1233, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '목숨', 'count': 2, 'genre': '댄스'}
{'index': {'_id': 1234, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'Super', 'count': 2, 'genre': '댄스'}
{'index': {'_id': 1235, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '신기루', 'count': 2, 'genre': '댄스'}
{'index': {'_id': 1236, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'ta', 'count': 2, 'genre': '댄스'}
{'index': {'_id': 1237, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'Fall', 'count': 2, 'genre': '발라드'}
{'index': {'_id': 1238, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '필요', 'count': 2, 'genre': '록/메탈'}
{'index': {'_id': 1239, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '감당', 'count': 2, 'genre': '랩/힙합'}
{'index': {'_id': 1240, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': 'tonight', 'count': 2, 'genre': '록/메탈'}
{'index': {'_id': 1241, '_type': 'k_b_

...

{'keyword': 'air', 'season': 'summer', 'count': 1}
{'index': {'_id': 6133, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'si', 'season': 'fall', 'count': 1}
{'index': {'_id': 6134, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'whoa', 'season': 'spring', 'count': 1}
{'index': {'_id': 6135, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'ba', 'season': 'fall', 'count': 1}
{'index': {'_id': 6136, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '홍대', 'season': 'winter', 'count': 1}
{'index': {'_id': 6137, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '인상', 'season': 'spring', 'count': 1}
{'index': {'_id': 6138, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '역', 'season': 'fall', 'count': 1}
{'index': {'_id': 6139, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'marry', 'season': 'summer', 'count': 1}
{'index': {'_id': 6140, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword

{'keyword': 'Touch', 'season': 'winter', 'count': 1}
{'index': {'_id': 6421, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Blah', 'season': 'summer', 'count': 1}
{'index': {'_id': 6422, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '구렁텅이', 'season': 'fall', 'count': 1}
{'index': {'_id': 6423, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Doctor', 'season': 'spring', 'count': 1}
{'index': {'_id': 6424, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Thunder', 'season': 'spring', 'count': 1}
{'index': {'_id': 6425, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Wuh', 'season': 'summer', 'count': 1}
{'index': {'_id': 6426, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'maze', 'season': 'spring', 'count': 1}
{'index': {'_id': 6427, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Mysterious', 'season': 'spring', 'count': 1}
{'index': {'_id': 6428, '_type': 'k_b_s', '_index': 'keyw

{'index': {'_id': 6734, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Dumb', 'season': 'spring', 'count': 1}
{'index': {'_id': 6735, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '소녀', 'season': 'winter', 'count': 1}
{'index': {'_id': 6736, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '캅', 'season': 'fall', 'count': 1}
{'index': {'_id': 6737, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Zone', 'season': 'fall', 'count': 1}
{'index': {'_id': 6738, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Got', 'season': 'spring', 'count': 1}
{'index': {'_id': 6739, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'YEAR', 'season': 'fall', 'count': 1}
{'index': {'_id': 6740, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '주눅', 'season': 'winter', 'count': 1}
{'index': {'_id': 6741, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'anxiety', 'season': 'spring', 'count': 1}
{'i

{'index': {'_id': 7049, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '우정', 'season': 'spring', 'count': 1}
{'index': {'_id': 7050, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'Need', 'season': 'summer', 'count': 1}
{'index': {'_id': 7051, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '굳은살', 'season': 'spring', 'count': 1}
{'index': {'_id': 7052, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '실패', 'season': 'winter', 'count': 1}
{'index': {'_id': 7053, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '군', 'season': 'summer', 'count': 1}
{'index': {'_id': 7054, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '유령', 'season': 'summer', 'count': 1}
{'index': {'_id': 7055, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'FOREVER', 'season': 'winter', 'count': 1}
{'index': {'_id': 7056, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '끄', 'season': 'spring', 'count': 1}
{'

{'keyword': '맨', 'season': 'fall', 'count': 1}
{'index': {'_id': 7362, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '기계', 'season': 'spring', 'count': 1}
{'index': {'_id': 7363, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': 'music', 'season': 'summer', 'count': 1}
{'index': {'_id': 7364, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '제때', 'season': 'spring', 'count': 1}
{'index': {'_id': 7365, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '가을빛', 'season': 'fall', 'count': 1}
{'index': {'_id': 7366, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '로동', 'season': 'winter', 'count': 1}
{'index': {'_id': 7367, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '바캉스', 'season': 'summer', 'count': 1}
{'index': {'_id': 7368, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keyword': '마라톤', 'season': 'spring', 'count': 1}
{'index': {'_id': 7369, '_type': 'k_b_s', '_index': 'keyword_by_season'}}
{'keywo

In [12]:
# 100번 이상 나온 단어만 json 파일로 저장 

# 장르별 키워드 json 파일에 저장

cnt = 0

genre_count = genre_count.filter(col("count") > 10)
result = rec_to_actions(genre_count.toPandas(),"keyword_by_genre", "k_b_g", cnt)
    
dest_file = 'elk_keyword_2/keyword_by_genre.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")
    

# 시대별 키워드 json 파일에 저장

cnt =0
age_count = age_count.filter(col("count") > 100)
result = rec_to_actions(age_count.toPandas(),"keyword_by_age", "k_b_a", cnt)

dest_file = 'elk_keyword_2/keyword_by_age.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")
    
# 연도별 키워드 json 파일에 저장

cnt =0
one_year_count = one_year_count.filter(col("count") > 100)
result = rec_to_actions(one_year_count.toPandas(),"keyword_by_year", "k_b_y", cnt)
    
dest_file = 'elk_keyword_2/keyword_by_year.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")

# 계절별 키워드 json 파일에 저장
season_count = season_count.filter(col("count") > 100)
result = rec_to_actions(season_count.toPandas(),"keyword_by_season", "k_b_s", cnt)
    
dest_file = 'elk_keyword_2/keyword_by_season.json'
output_file = open(dest_file, 'w', encoding='utf-8')
for l in result:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")


0
{'index': {'_id': 0, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사랑', 'count': 512, 'genre': '발라드'}
{'index': {'_id': 1, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '말', 'count': 322, 'genre': '발라드'}
{'index': {'_id': 2, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '날', 'count': 301, 'genre': '발라드'}
{'index': {'_id': 3, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사람', 'count': 151, 'genre': '발라드'}
{'index': {'_id': 4, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '눈물', 'count': 141, 'genre': '발라드'}
{'index': {'_id': 5, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '때', 'count': 131, 'genre': '발라드'}
{'index': {'_id': 6, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '사랑', 'count': 128, 'genre': '댄스'}
{'index': {'_id': 7, '_type': 'k_b_g', '_index': 'keyword_by_genre'}}
{'keyword': '맘', 'count': 122, 'genre': '발라드'}
{'index': {'_id': 8, '_type': 'k_b_g', '_index': 'keyword_b

KeyboardInterrupt: 

In [ ]:
# 장르별 키워드 json 파일에 저장


lst = []
cnt = 0
g = key_genre_df.groupby("genre").count()
genre_list = g.select("genre").rdd.flatMap(lambda x: x).collect()

for genre in genre_list:
    temp_count = genre_count.filter(col("genre") == genre)
    temp_count.show()
    temp_count.toPandas().to_json('wordcount_result/wordcount_by_genre_'+str(genre.split("/")[0])+'.json', force_ascii=False, orient="records")
    
    result = rec_to_actions(temp_count.toPandas(),"keyword_by_genre", "k_b_g", cnt)
    lst += result
    cnt += len(result)
    
dest_file = 'elk_keyword/keyword_by_genre.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in lst:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")



In [ ]:
# 시대별 키워드 json 파일에 저장

lst =[]
cnt =0


for age in ["2011-2013","2014-2016","2017-2019"]:
    temp_count = year_count.filter(col("age") == age)
    temp_count.show()
    temp_count.toPandas().to_json('wordcount_result/keyword_by_age_'+str(age)+'.json', force_ascii=False, orient="records")
    
    result = rec_to_actions(temp_count.toPandas(),"keyword_by_age", "k_b_a", cnt)
    lst += result
    cnt += len(result)
    
dest_file = 'elk_keyword/keyword_by_age.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in lst:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")


In [ ]:
#연도별 키워드 json 파일에 저장

lst = []
cnt = 0

for year in range(2011,2021):
    temp_count = one_year_count.filter(col("year") == str(year))
    #temp_count.show()
    #temp_count.toPandas().to_json('wordcount_result/wordcount_by_year_'+str(year)+'.json', force_ascii=False, orient="records")

    result = rec_to_actions(temp_count.toPandas(),"keyword_by_year", "k_b_y", cnt)
    lst += result
    cnt += len(result)
    
dest_file = 'elk_keyword/keyword_by_year.json'
output_file = open(dest_file, 'w', encoding='utf-8')

for l in lst:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")

In [ ]:
#계절 별 키워드 json 파일에 저장

#계절 별 키워드 json 파일에 저장
seasons = ["spring", "summer", "fall", "winter"]

lst = [] 

cnt = 0
for season in seasons:
    temp_count = season_count.filter(col("season") == season).filter(col("count") > 100)
    
    #temp_count.show()
    #temp_count.toPandas().to_json('wordcount_result/wordcount_by_season_'+str(season)+'.json', force_ascii=False, orient="records")
    
    result = rec_to_actions(temp_count.toPandas(),"keyword_by_season", "k_b_s", cnt)
    lst += result
    cnt += len(result)
    
dest_file = 'elk_keyword/keyword_by_season.json'
output_file = open(dest_file, 'w', encoding='utf-8')
for l in lst:
    print(eval(l))
    json.dump(eval(l), output_file, ensure_ascii=False) 
    output_file.write("\n")

    
"""
seasons = ["spring", "summer", "fall", "winter"]
for season in seasons:
    temp_count = season_count.filter(col("season") == season)
    temp_count.show()
    temp_count.toPandas().to_json('wordcount_result/wordcount_by_season_'+str(season)+'.json', force_ascii=False, orient="records")
    
"""



In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# Datetime, String, Interger Example Dataframe 

listDate = ['2020-01-01 00:00:00','2020-01-01 00:01:00','2020-01-01 00:02:00', '2020-01-01 00:03:00']
listStrings = ['a','b','c','d']
listInterger = [1, 2, 3, 4 ]

df = pd.DataFrame([ x for x in zip(listDate,listStrings,listInterger)], columns=['date','string', 'interger'])
df['date'] = pd.to_datetime(df['date'])



# dataframe to elasticsearch

es = Elasticsearch(host='192.168.0.17', port='50099')
es.indices.create(index='bulk_test',body={})

df = pd.DataFrame(data = {'date' : df['date'],
                          'strings': df['string'],
                          'interger' : df['interger']})
 
documents = df.to_dict(orient='records')
bulk(es, documents, index='bulk_test',doc_type='foo', raise_on_error=True)